In [0]:
import logging as log
import datetime

2023-04-06 04:05:34,715 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


In [0]:
def get_log_level(level):
    level = level.lower()
    if  level == "info":
        return log.INFO
    elif level == "debug":
        return log.DEBUG
    elif level == "trace":
        return log.CRITICAL

# initialize log
def init_log(config_data):
    try:
        log_format = f"%(asctime)s [%(levelname)s] -> [%(threadName)s] - %(funcName)s - %(lineno)d - %(message)s "
        log_formatter = log.Formatter(log_format)

        root_logger = log.getLogger()
        root_logger.setLevel(get_log_level(config_data.get('logging').get('mode')))
        if root_logger.handlers:
            root_logger.handlers.clear()

        console_handler = log.StreamHandler()
        console_handler.setLevel(get_log_level(config_data.get('logging').get('mode')))
        console_handler.setFormatter(log_formatter)
        root_logger.addHandler(console_handler)

        file_date = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        log_dir = config_data.get('logging').get('folder_path')
        log_file_name = f"/{config_data.get('logging').get('file_name')}_{file_date}.log"
        log_path = f"{log_dir}{log_file_name}"
        log.info(log_path)
#         log.basicConfig(level=log.INFO, filename=log_path, filemode="a",
#                         format=log_format)
        
        file_handler = log.FileHandler(log_path,mode='a')
        file_handler.setFormatter(log_formatter)
        root_logger.addHandler(file_handler)
    except Exception as e:
        raise Exception('log initialization failed: %s' % e)

In [0]:
# initialize configuration
def init_config(config_path):
    try:
        """read configuration from json fle"""
        config_data = spark.read.option("multiline","true").json(config_path).collect()[0]
        config_data = config_data.asDict(True)
#         """hardcoded"""
#         config_data = {'job_num': {'ods_job_num': 3}, 'logging': {'file_name': 'parquet', 'folder_path': '/tmp', 'mode': 'info'}, 'source': {'catalog_nm':'`omi-catalog`','audit_schema_nm': '`omi-catalog`.si_core_audit', 'jdbc_offset': 100000, 'database': 'postgres', 'jdbc_fetch_size': 10000,'delta_partitions': 10,'jdbc_partitions': 10, 'driver': 'org.postgresql.Driver', 'password': 'Sapiens2022', 'url': 'jdbc:postgresql://omi-tia-ods-perftest-cluster.cluster-cq2voeb9fipc.eu-west-1.rds.amazonaws.com:5432/tiadr', 'user': 'postgres'}}
        init_log(config_data)
        return config_data
    except Exception as e:
        raise Exception('config initialization failed: %s' % e)

2023-04-06 04:05:34,817 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 


In [0]:
# get dataframe from relational db using jdbc connection
def get_df(config_data,query):
    try:
#         dataframe = spark.read.jdbc(url=jdbcUrl, table=query, properties=jdbcProperties)
        dataframe = spark.read.format("jdbc") \
                .option("url", config_data.get('source').get('url')) \
                .option("driver", config_data.get('source').get('driver')) \
                .option("query", query) \
                .option("user", config_data.get('source').get('user')) \
                .option("password", config_data.get('source').get('password')) \
                .option("fetchsize",config_data.get('source').get('jdbc_fetch_size')) \
                .load()
        return dataframe
    except Exception as e:
        print('exception in get_df: %s' % e)

In [0]:
def exec_spark_sql(query,config_data=None):
    try:
        df = spark.sql(query)
        return df
    except Exception as e:
        raise Exception(f'exception in exec_spark_sql: {e} : query: {query}')

2023-04-06 04:05:35,018 [INFO] -> [Thread-6] - wait_for_commands - 575 - Received command c on object id p0 
